# # Project: NLP to Analyze Sentiments of Consumer Reviews of Hotels    

In this project, we use Sentiment Analysis of NLP to analyze reviews of hotels. We would like to understand if a review is positive or negative (without looking at the ratings). We will use the following libraries:
- NLTK: the most famous python module for NLP techniques
- Gensim: a topic-modelling and vector space modelling toolkit
- Scikit-learn: the most used python machine learning library

The dataset is from Kaggle and contains 515,000 customer reviews and scoring of 1493 luxury hotels offered in Booking.come across Europe.
The dataset can be found here: https://www.kaggle.com/jiashenliu/515k-hotel-reviews-data-in-europe/version/1

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Part 1: Load the Data and Take a Look

In [17]:
# load the data into a datadrame
reviews = pd.read_csv('/Users/Amir/PythonProjects/HotelRRDataset/Hotel_Reviews.csv')
reviews.info()

# append positive and negative reviews to have all in one column (in booking, there is two entires for 
# consumers to write positive and negative comments)
reviews["review"] = reviews["Negative_Review"] + reviews["Positive_Review"]

# We define a bad review as the one which has overall rating of lower than 5. We label them using actual ratings 
# to compare them to our predicion to evaluate the performance 
reviews["bad review?"] = reviews["Average_Score"].apply(lambda x: 1 if x < 5 else 0)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
Hotel_Address                                 515738 non-null object
Additional_Number_of_Scoring                  515738 non-null int64
Review_Date                                   515738 non-null object
Average_Score                                 515738 non-null float64
Hotel_Name                                    515738 non-null object
Reviewer_Nationality                          515738 non-null object
Negative_Review                               515738 non-null object
Review_Total_Negative_Word_Counts             515738 non-null int64
Total_Number_of_Reviews                       515738 non-null int64
Positive_Review                               515738 non-null object
Review_Total_Positive_Word_Counts             515738 non-null int64
Total_Number_of_Reviews_Reviewer_Has_Given    515738 non-null int64
Reviewer_Score                                515738 non-null flo

In [15]:
# Select 

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng,review,bad review?
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,I am so angry that i made this post available...,0
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968,No Negative No real complaints the hotel was g...,0
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968,Rooms are nice but for elderly a bit difficul...,0


## Part 2: 

## Part 3: 

In [1]:
# Drop unwante columns and return results in the same dataframe
#features_to_remove = ['address','attributes','business_id','categories','city','hours','is_open','latitude','longitude','name','neighborhood','postal_code','state','time']
# axis=1 lets Pandas know we want to drop columns, not rows
# inplace=True lets us drop the columns right here in our DataFrame, instead of returning a new DataFrame
#df.drop(labels=features_to_remove, axis=1, inplace=True)
#df.info()

In [3]:
# check for missing values (if any, they will prevent LR from running)
#df.isna().any()
# replace all of our NaNs with 0s
#df.fillna({'weekday_checkins':0,'weekend_checkins':0, 'average_tip_length':0, 'number_tips':0, 'average_caption_length':0,'number_pics':0}, inplace=True)
# check for missing values
#df.isna().any()

## Part 4: 

## Part 5: 

## Part 6: 